In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Define paths
sos_dir = 'data/sos'
non_sos_dir = 'data/non_sos'

# Function to load images
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=(64, 64))  # Resize image to 64x64
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return images, labels

# Load SOS and non-SOS images
sos_images, sos_labels = load_images_from_folder(sos_dir, 1)
non_sos_images, non_sos_labels = load_images_from_folder(non_sos_dir, 0)

# Combine and split the data
X = np.array(sos_images + non_sos_images)
y = np.array(sos_labels + non_sos_labels)
X = X / 255.0  # Normalize images
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Save the model
model.save('sos_gesture_model.h5')


In [ ]:
import cv2
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('sos_gesture_model.h5')

# Function to preprocess frame for model
def preprocess_frame(frame):
    frame = cv2.resize(frame, (64, 64))  # Resize frame to 64x64
    frame = frame / 255.0  # Normalize
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    return frame

# Initialize video capture
cap = cv2.VideoCapture(0)  # Use the correct camera index or video file

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame and predict
    preprocessed_frame = preprocess_frame(frame)
    prediction = model.predict(preprocessed_frame)
    if prediction[0] > 0.5:
        label = "SOS Gesture Detected!"
    else:
        label = "No SOS Gesture"

    # Display the result
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Video Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
